In [1]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_csv('processed_data.csv')

In [6]:
df

,OPEN_ORG_NUM,IDF_TYP_CD,GENDER,bad_good,LAST_OPEN_TENURE_DAYS,G_OS_PRCP_SUM,OS_PRCP_SUM_THREE,OS_PRCP_SUM_SIX,guozhai_flag,L3_DR_AMT,...,CHANNEL_AUTO_DTAIN_CNT,CHANNEL_AUTO_DOUTTA_CNT,L3_CHANNEL_AUTO_DTAIN_AVGAMT,L3_CHANNEL_AUTO_DOUTTA_AVGAMT,L3_CHANNEL_AUTO_DTAIN_AVGCNT,L3_CHANNEL_AUTO_DOUTTA_AVGCNT,L6_CHANNEL_AUTO_DTAIN_AVGAMT,L6_CHANNEL_AUTO_DOUTTA_AVGAMT,L6_CHANNEL_AUTO_DTAIN_AVGCNT,L6_CHANNEL_AUTO_DOUTTA_AVGCNT
0,0.116279,0.0,1.0,0.0,0.000000,0.007875,0.007416,0.007416,0.0,0.004431,...,0.0,0.000000,0.003836,0.000000,0.000347,0.000000,0.002721,0.000000,0.000176,0.000000
1,0.058256,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000230,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.058256,0.0,1.0,1.0,0.068182,0.000000,0.017500,0.017500,0.0,0.002054,...,0.0,0.019417,0.000000,0.001159,0.000000,0.007275,0.000000,0.001588,0.000000,0.004464
3,0.058256,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.058256,0.0,0.0,0.0,0.295455,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001717,0.000000,0.001594
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285280,0.233953,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
285281,0.593023,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
285282,0.002209,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000129,0.000000,0.001984,0.000000,0.000172,0.000000,0.001276
285283,0.337791,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.001160,0.000000,0.000693,0.000000,0.000823,0.000000,0.000352,0.000000


In [7]:
cor_matrix = df.corr()

In [9]:
import pickle
f = open("cor_matrix.pkl", 'wb')
pickle.dump(cor_matrix, f)

In [19]:
df['OS_PRCP_SUM_SIX']

0         0.007416
1         0.000000
2         0.017500
3         0.000000
4         0.000000
            ...   
285280    0.000000
285281    0.000000
285282    0.000000
285283    0.000000
285284    0.000000
Name: OS_PRCP_SUM_SIX, Length: 285285, dtype: float64

In [56]:
data_selected = []
for i in list(cor_matrix.columns.values):
    if abs(cor_matrix[i]['bad_good'])>=0.05:
        data_selected.append(i)

In [57]:
data_great=df.loc[:, data_selected]
data_great.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285285 entries, 0 to 285284
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   bad_good                 285285 non-null  float64
 1   DEP_SA_OPEN_TENURE_DAYS  285285 non-null  float64
 2   DEP_SA_LAST_TENURE_DAYS  285285 non-null  float64
 3   DEP_SA_AVG_TENURE_DAYS   285285 non-null  float64
 4   LOAN_FLAG                285285 non-null  float64
 5   OS_PRCP_SUM_SIX          285285 non-null  float64
dtypes: float64(6)
memory usage: 13.1 MB


In [59]:
outlier_feature=list(data_great.columns)
outlier_feature.remove('bad_good')
for i in outlier_feature:
    mean=data_great[i].mean()
    std=data_great[i].std()
    upper_limit=mean+3*std
    lower_limit=mean-3*std
    condition=(data_great[i]>upper_limit)|(data_great[i]<lower_limit)
    print('outlier numbers:%.2f'%len(data_great.loc[condition]))
    print(i+' outlier number rate:%.6f'%(len(data_great.loc[condition])/len(data_great)))
    print('-'*40)

outlier numbers:1790.00
DEP_SA_OPEN_TENURE_DAYS outlier number rate:0.006274
----------------------------------------
outlier numbers:3332.00
DEP_SA_LAST_TENURE_DAYS outlier number rate:0.011680
----------------------------------------
outlier numbers:2333.00
DEP_SA_AVG_TENURE_DAYS outlier number rate:0.008178
----------------------------------------
outlier numbers:13651.00
LOAN_FLAG outlier number rate:0.047850
----------------------------------------
outlier numbers:1594.00
OS_PRCP_SUM_SIX outlier number rate:0.005587
----------------------------------------


In [60]:
#根据特征处理异常值
data_filtered = data_great.copy()
for i in data_selected:
    if(i == 'bad_good'):
        continue
    mean=data_filtered[i].mean()
    std=data_filtered[i].std()
    upper_limit=mean+3*std
    lower_limit=mean-3*std
    condition=(data_filtered[i]<=upper_limit)&(data_filtered[i]>=lower_limit)
    data_filtered=data_filtered.loc[condition].copy()
data_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 267296 entries, 1 to 285284
Data columns (total 6 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   bad_good                 267296 non-null  float64
 1   DEP_SA_OPEN_TENURE_DAYS  267296 non-null  float64
 2   DEP_SA_LAST_TENURE_DAYS  267296 non-null  float64
 3   DEP_SA_AVG_TENURE_DAYS   267296 non-null  float64
 4   LOAN_FLAG                267296 non-null  float64
 5   OS_PRCP_SUM_SIX          267296 non-null  float64
dtypes: float64(6)
memory usage: 14.3 MB


In [61]:
# 重新归一化
data_filtered = (data_filtered-data_filtered.min())/(data_filtered.max()-data_filtered.min())

In [62]:
data_great.corr()['bad_good'].abs().sort_values(ascending=False)

bad_good                   1.000000
LOAN_FLAG                  0.588719
DEP_SA_OPEN_TENURE_DAYS    0.163460
DEP_SA_AVG_TENURE_DAYS     0.156396
DEP_SA_LAST_TENURE_DAYS    0.132562
OS_PRCP_SUM_SIX            0.004679
Name: bad_good, dtype: float64